### Load and transform data
<br>
36-1 Okiru
<br>
7-1 Linux.Mirai

In [1]:
import os
import pandas as pd

dir = 'raw/'

# read all file names in the directory
file_names = os.listdir(dir)

df = []

# for all files in the directory
for i in range(len(file_names)):
    # print file name
    print(file_names[i])
    # read the file
    df_temp = pd.read_csv(dir + file_names[i], sep='\t', skiprows=8, header=None, low_memory=False)
    # append the file to the list
    df.append(df_temp)
    # print the shape
    print(df_temp.shape)
    print()

CTU-IoT-Malware-Capture-36-1.labeled
(13645099, 21)

CTU-IoT-Malware-Capture-7-1.labeled
(11454715, 21)



In [2]:
# for df's in df
for i in range(len(df)):
    # add column names to the dataframes
    df[i].columns = ['ts', 'uid', 'id.orig_h', 'id.orig_p', 'id.resp_h', 'id.resp_p', 
                     'proto', 'service', 'duration', 'orig_bytes', 'resp_bytes', 
                     'conn_state', 'local_orig', 'local_resp', 'missed_bytes', 
                     'history', 'orig_pkts', 'orig_ip_bytes', 'resp_pkts', 'resp_ip_bytes', 'label']
    
df[0].head()

,ts,uid,id.orig_h,id.orig_p,id.resp_h,id.resp_p,proto,service,duration,orig_bytes,...,conn_state,local_orig,local_resp,missed_bytes,history,orig_pkts,orig_ip_bytes,resp_pkts,resp_ip_bytes,label
0,1545395814.964457,CNsbYd1o5m4QJW88Rj,192.168.1.198,54118.0,185.130.215.13,57722.0,tcp,-,3.141696,0,...,S0,-,-,0.0,S,3.0,180.0,0.0,0.0,- Malicious C&C-HeartBeat
1,1545395814.965207,Civxru33RoY9pFbKOc,192.168.1.198,36097.0,105.49.30.168,37215.0,tcp,-,-,-,...,S0,-,-,0.0,S,1.0,40.0,0.0,0.0,- Malicious Okiru
2,1545395814.965212,C1rCRiPW2VkdO4W83,192.168.1.198,36097.0,18.206.164.168,37215.0,tcp,-,-,-,...,S0,-,-,0.0,S,1.0,40.0,0.0,0.0,- Malicious Okiru
3,1545395814.965214,CVaptz3TPiGtelf6N7,192.168.1.198,36097.0,181.237.153.227,37215.0,tcp,-,-,-,...,S0,-,-,0.0,S,1.0,40.0,0.0,0.0,- Malicious Okiru
4,1545395814.965216,CxGWPH3GVTrdfmggFl,192.168.1.198,36097.0,182.150.31.187,37215.0,tcp,-,-,-,...,S0,-,-,0.0,S,1.0,40.0,0.0,0.0,- Malicious Okiru


Organize labels

In [3]:
# check the number of nan values in label column 
print(df[0]['label'].isna().sum())
print(df[1]['label'].isna().sum())

1
1


In [4]:
# remove the rows that has nan values in label column
df[0] = df[0][df[0]['label'].notna()]
df[1] = df[1][df[1]['label'].notna()]

# print the unique values of the label column
print(df[0]['label'].unique())
print(df[1]['label'].unique())

['-   Malicious   C&C-HeartBeat' '-   Malicious   Okiru' '-   Benign   -'
 '-   Malicious   Okiru-Attack']
['(empty)   Benign   -' '(empty)   Malicious   C&C-HeartBeat'
 '(empty)   Malicious   Okiru' '(empty)   Malicious   DDoS'
 'COLnd035cNITygYHp3   Benign   -' 'CARhxZ3hLNVO3xYFok   Benign   -']


In [5]:
# add a new column called target with value 0
df[0]['target'] = 0
df[1]['target'] = 0

# label the malicious rows
df[0].loc[df[0]['label'].str.contains('Malicious'), 'target'] = 1
df[1].loc[df[1]['label'].str.contains('Malicious'), 'target'] = 2

# drop the label column
df[0].drop('label', axis=1, inplace=True)
df[1].drop('label', axis=1, inplace=True)

# concat the dataframes to df_malicious
df_all = pd.concat([df[0], df[1]], ignore_index=True)

# print the value counts of target column
print(df_all['target'].value_counts())

C:\Users\Stone\AppData\Local\Temp\ipykernel_12684\1273909772.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[1]['target'] = 0
C:\Users\Stone\AppData\Local\Temp\ipykernel_12684\1273909772.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[1].drop('label', axis=1, inplace=True)


1    13642435
2    11378759
0       78618
Name: target, dtype: int64


Remove timestamp and id

In [6]:
# remove first 2 columns
df_all = df_all.iloc[:, 2:]

pd.set_option('display.max_columns', None)
df_all.head()

,id.orig_h,id.orig_p,id.resp_h,id.resp_p,proto,service,duration,orig_bytes,resp_bytes,conn_state,local_orig,local_resp,missed_bytes,history,orig_pkts,orig_ip_bytes,resp_pkts,resp_ip_bytes,target
0,192.168.1.198,54118.0,185.130.215.13,57722.0,tcp,-,3.141696,0,0,S0,-,-,0.0,S,3.0,180.0,0.0,0.0,1
1,192.168.1.198,36097.0,105.49.30.168,37215.0,tcp,-,-,-,-,S0,-,-,0.0,S,1.0,40.0,0.0,0.0,1
2,192.168.1.198,36097.0,18.206.164.168,37215.0,tcp,-,-,-,-,S0,-,-,0.0,S,1.0,40.0,0.0,0.0,1
3,192.168.1.198,36097.0,181.237.153.227,37215.0,tcp,-,-,-,-,S0,-,-,0.0,S,1.0,40.0,0.0,0.0,1
4,192.168.1.198,36097.0,182.150.31.187,37215.0,tcp,-,-,-,-,S0,-,-,0.0,S,1.0,40.0,0.0,0.0,1


### Split nominal and numeric features

In [7]:
# print the value counts of column local_orig and local_resp
print(df_all['local_orig'].value_counts())
print(df_all['local_resp'].value_counts())

-    25099812
Name: local_orig, dtype: int64
-    25099812
Name: local_resp, dtype: int64


In [8]:
# drop the columns local_orig and local_resp
df_all.drop(['local_orig', 'local_resp'], axis=1, inplace=True)

df_all.head()

,id.orig_h,id.orig_p,id.resp_h,id.resp_p,proto,service,duration,orig_bytes,resp_bytes,conn_state,missed_bytes,history,orig_pkts,orig_ip_bytes,resp_pkts,resp_ip_bytes,target
0,192.168.1.198,54118.0,185.130.215.13,57722.0,tcp,-,3.141696,0,0,S0,0.0,S,3.0,180.0,0.0,0.0,1
1,192.168.1.198,36097.0,105.49.30.168,37215.0,tcp,-,-,-,-,S0,0.0,S,1.0,40.0,0.0,0.0,1
2,192.168.1.198,36097.0,18.206.164.168,37215.0,tcp,-,-,-,-,S0,0.0,S,1.0,40.0,0.0,0.0,1
3,192.168.1.198,36097.0,181.237.153.227,37215.0,tcp,-,-,-,-,S0,0.0,S,1.0,40.0,0.0,0.0,1
4,192.168.1.198,36097.0,182.150.31.187,37215.0,tcp,-,-,-,-,S0,0.0,S,1.0,40.0,0.0,0.0,1


| id.orig_h | id.orig_p | id.resp_h | id.resp_p | proto | service | duration | orig_bytes | resp_bytes |
|-----------|-----------|-----------|-----------|-------|---------|----------|------------|------------|
| addr      | port      | addr      | port      | enum  | string  | interval | count      | count      | 

<br>

| conn_state | missed_bytes | history | orig_pkts | orig_ip_bytes | resp_pkts | resp_ip_bytes |
|------------|--------------|---------|-----------|---------------|-----------|---------------|
| string     | count        | string  | count     | count         | count     | count         |

In [9]:
# df_num is column duration, orig_bytes, resp_bytes, missed_bytes, orig_pkts, orig_ip_bytes, resp_pkts, resp_ip_bytes
df_num = df_all[['duration', 'orig_bytes', 'resp_bytes', 'missed_bytes', 
                 'orig_pkts', 'orig_ip_bytes', 'resp_pkts', 'resp_ip_bytes']]

# df_cat is the rest of the columns
df_cat = df_all.drop(['duration', 'orig_bytes', 'resp_bytes', 'missed_bytes', 
                      'orig_pkts', 'orig_ip_bytes', 'resp_pkts', 'resp_ip_bytes'], axis=1)

In [10]:
# print the number of missing values in each column of df_num
print(df_num.isna().sum())

duration         0
orig_bytes       0
resp_bytes       0
missed_bytes     0
orig_pkts        0
orig_ip_bytes    0
resp_pkts        0
resp_ip_bytes    0
dtype: int64


In [11]:
# print the number of value contains string '-' for each column of df_cat
print((df_cat == '-').sum())

id.orig_h            0
id.orig_p            0
id.resp_h            0
id.resp_p            0
proto                0
service       25099051
conn_state           0
history           7410
target               0
dtype: int64


In [12]:
# drop service column
df_cat.drop('service', axis=1, inplace=True)

### Encoding nominal features

IP address

In [13]:
# check if there are any ipv6 addresses
print(df_cat['id.orig_h'].str.contains(':').sum())
print(df_cat['id.resp_h'].str.contains(':').sum())

0
0


In [14]:
# split column id.orig_h into 4 columns by .
df_cat[['id.orig_h_1', 'id.orig_h_2', 'id.orig_h_3', 'id.orig_h_4']] = df_cat['id.orig_h'].str.split('.', expand=True)

# split column id.resp_h
df_cat[['id.resp_h_1', 'id.resp_h_2', 'id.resp_h_3', 'id.resp_h_4']] = df_cat['id.resp_h'].str.split('.', expand=True)

# drop columns id.orig_h and id.resp_h
df_cat.drop(['id.orig_h', 'id.resp_h'], axis=1, inplace=True)

In [15]:
df_cat.head()

,id.orig_p,id.resp_p,proto,conn_state,history,target,id.orig_h_1,id.orig_h_2,id.orig_h_3,id.orig_h_4,id.resp_h_1,id.resp_h_2,id.resp_h_3,id.resp_h_4
0,54118.0,57722.0,tcp,S0,S,1,192,168,1,198,185,130,215,13
1,36097.0,37215.0,tcp,S0,S,1,192,168,1,198,105,49,30,168
2,36097.0,37215.0,tcp,S0,S,1,192,168,1,198,18,206,164,168
3,36097.0,37215.0,tcp,S0,S,1,192,168,1,198,181,237,153,227
4,36097.0,37215.0,tcp,S0,S,1,192,168,1,198,182,150,31,187


In [16]:
# print the number of missing values in first two columns of df_cat
print(df_cat.iloc[:, :2].isna().sum())

id.orig_p    0
id.resp_p    0
dtype: int64


Labeling port numbers
well-known ports (0-1023), registered ports (1024-49151), and dynamic/private ports (49152-65535)

In [17]:
# convert the first 2 columns to numeric
df_cat.iloc[:, :2] = df_cat.iloc[:, :2].apply(pd.to_numeric)
# df_malicious[['id.orig_p', 'id.resp_p']] = df_malicious[['id.orig_p', 'id.resp_p']].apply(pd.to_numeric)
# df_benign[['id.orig_p', 'id.resp_p']] = df_benign[['id.orig_p', 'id.resp_p']].apply(pd.to_numeric)

# print the data types of first 2 columns
print(df_cat.iloc[:, :2].dtypes)

id.orig_p    float64
id.resp_p    float64
dtype: object


In [18]:
def label_port(port):
    if 0 <= port <= 1023:
        return 'wk'
    elif 1024 <= port <= 49151:
        return 'reg'
    elif 49152 <= port <= 65535:
        return 'dyn'
    else:
        return 'unk'

# Assuming df_malicious and df_benign are your dataframes
df_cat['id.orig_p'] = df_cat['id.orig_p'].apply(label_port)
df_cat['id.resp_p'] = df_cat['id.resp_p'].apply(label_port)

# print the unique values of id.orig_p and id.resp_p
print(df_cat['id.orig_p'].unique())
print(df_cat['id.resp_p'].unique())


['dyn' 'reg' 'wk']
['dyn' 'reg' 'wk']


Analysis and label reduction before one-hot encoding

In [19]:
df_cat.head()

,id.orig_p,id.resp_p,proto,conn_state,history,target,id.orig_h_1,id.orig_h_2,id.orig_h_3,id.orig_h_4,id.resp_h_1,id.resp_h_2,id.resp_h_3,id.resp_h_4
0,dyn,dyn,tcp,S0,S,1,192,168,1,198,185,130,215,13
1,reg,reg,tcp,S0,S,1,192,168,1,198,105,49,30,168
2,reg,reg,tcp,S0,S,1,192,168,1,198,18,206,164,168
3,reg,reg,tcp,S0,S,1,192,168,1,198,181,237,153,227
4,reg,reg,tcp,S0,S,1,192,168,1,198,182,150,31,187


In [20]:
# print the value counts of proto, conn_state, history 
print(df_cat['proto'].value_counts())
print()
print(df_cat['conn_state'].value_counts())
print()
print(df_cat['history'].value_counts())

tcp     24981741
udp       110661
icmp        7410
Name: proto, dtype: int64

S0        25073118
REJ          15797
OTH           7429
SF            2978
RSTO           311
RSTOS0         132
RSTR            19
S2              11
SHR             11
S1               4
SH               2
Name: conn_state, dtype: int64

S             24965435
D               107684
Sr               15792
-                 7410
Dd                2968
ShR                308
R                  120
^d                  12
SaR                 12
DdA                  8
ShADF                7
ShAFr                7
ShADFr               7
Hr                   5
ShADadfF             5
ShA                  3
ShADr                3
A                    3
ShAF                 3
F                    2
ShADFrfR             2
ShwR                 2
ShADFdRtf            2
DadA                 2
ShADad               1
DdAf                 1
DdAa                 1
^ha                  1
ShADFadRfR           1
ShADadfFR     

In [21]:
# if the value contains string 'RST' in column conn_state, replace it with 'RST'
df_cat.loc[df_cat['conn_state'].str.contains('RST'), 'conn_state'] = 'RST'

# if the value contains string 'SH' in column conn_state, replace it with 'SH'
df_cat.loc[df_cat['conn_state'].str.contains('SH'), 'conn_state'] = 'SH'

# if the value is 'SF' in column conn_state, replace it with 'S0
df_cat.loc[df_cat['conn_state'] == 'SF', 'conn_state'] = 'S0'

In [22]:
# count the unique values in the target column that has value '-' in history column
print(df_cat.loc[df_cat['history'] == '-', 'target'].value_counts())

0    7410
Name: target, dtype: int64


Connection history
| Letter | Meaning                                      |
|--------|----------------------------------------------|
| s      | a SYN w/o the ACK bit set                    |
| h      | a SYN+ACK (“handshake”)                      |
| a      | a pure ACK                                   |
| d      | packet with payload (“data”)                 |
| f      | packet with FIN bit set                      |
| r      | packet with RST bit set                      |
| c      | packet with a bad checksum (applies to UDP)  |
| g      | a content gap                                |
| t      | packet with retransmitted payload            |
| w      | packet with a zero window advertisement      |
| i      | inconsistent packet (e.g., FIN+RST bits set) |
| q      | multi-flag packet (SYN+FIN or SYN+RST bits set) |
| ^      | connection direction was flipped by Zeek’s heuristic |

In [23]:
# count the values in the target column contains string '^' in history column
# this might indicate that the connection doesn't follow the typical patterns of established protocols
print(df_cat.loc[df_cat['history'].str.contains('\^'), 'target'].value_counts())

0    13
Name: target, dtype: int64


In [24]:
# count the values in the target column contains string 'i' or 'I' in history column
# packets that are inconsistent (e.g., FIN+RST bits set) might suggest attempts to disrupt or manipulate a connection
print(df_cat.loc[df_cat['history'].str.contains('i|I'), 'target'].value_counts())

Series([], Name: target, dtype: int64)


In [25]:
# count the values in the target column contains string 'r' or 'R' in history column
# while RST is a normal part of TCP, excessive resets or resets in unusual contexts can indicate connection aborts often seen in malicious activities.
print(df_cat.loc[df_cat['history'].str.contains('r|R'), 'target'].value_counts())

2    15547
1      592
0      124
Name: target, dtype: int64


In [26]:
# if value in history column contains string 'r' or 'R', replace it with 'r'
df_cat.loc[df_cat['history'].str.contains('r|R'), 'history'] = 'r'

In [27]:
# count the values in the target column contains string 'd' or 'D' in history column
print(df_cat.loc[df_cat['history'].str.contains('d|D'), 'target'].value_counts())

0    71082
2    39608
1        1
Name: target, dtype: int64


In [28]:
# if value in history column contains string 'd' or 'D', replace it with 'd'
df_cat.loc[df_cat['history'].str.contains('d|D'), 'history'] = 'd'

In [29]:
# count the values in the target column contains string 'c' or 'C' in history column
print(df_cat.loc[df_cat['history'].str.contains('c|C'), 'target'].value_counts())

Series([], Name: target, dtype: int64)


In [30]:
# count the values in the target column contains string 'q' or 'Q' in history column
print(df_cat.loc[df_cat['history'].str.contains('q|Q'), 'target'].value_counts())

Series([], Name: target, dtype: int64)


In [31]:
# count the values in the target column contains string 't' or 'T' in history column
print(df_cat.loc[df_cat['history'].str.contains('t|T'), 'target'].value_counts())

Series([], Name: target, dtype: int64)


In [32]:
# count the values in the target column contains string 'h' or 'H' in history column
print(df_cat.loc[df_cat['history'].str.contains('h|H'), 'target'].value_counts())

2    6
0    2
Name: target, dtype: int64


In [33]:
# if value in history column contains string 'h' or 'H', replace it with 'h'
df_cat.loc[df_cat['history'].str.contains('h|H'), 'history'] = 'h'

In [34]:
# count the values in the target column contains string 'f' or 'F' in history column
print(df_cat.loc[df_cat['history'].str.contains('f|F'), 'target'].value_counts())

2    2
Name: target, dtype: int64


In [35]:
# count the values in the target column contains string 'a' or 'A' in history column
print(df_cat.loc[df_cat['history'].str.contains('a|A'), 'target'].value_counts())

2    3
Name: target, dtype: int64


Review

In [36]:
# # print the value counts of proto, conn_state, history 
print(df_cat['conn_state'].value_counts())
print()
print(df_cat['history'].value_counts())

S0     25076096
REJ       15797
OTH        7429
RST         462
SH           13
S2           11
S1            4
Name: conn_state, dtype: int64

S    24965435
d      110691
r       16263
-        7410
h           8
A           3
F           2
Name: history, dtype: int64


In [37]:
# if the value contains string 'h', 'A', 'F' in column history, replace it with 'B'
df_cat.loc[df_cat['history'].str.contains('h|A|F'), 'history'] = 'B'

# if the value is 'SH', 'S1', 'S2' in column conn_state, replace it with 'N'
df_cat.loc[df_cat['conn_state'].str.contains('SH|S1|S2'), 'conn_state'] = 'N'

# print the value counts of proto, conn_state, history 
print(df_cat['conn_state'].value_counts())
print()
print(df_cat['history'].value_counts())

S0     25076096
REJ       15797
OTH        7429
RST         462
N            28
Name: conn_state, dtype: int64

S    24965435
d      110691
r       16263
-        7410
B          13
Name: history, dtype: int64


In [38]:
df_cat.head()

,id.orig_p,id.resp_p,proto,conn_state,history,target,id.orig_h_1,id.orig_h_2,id.orig_h_3,id.orig_h_4,id.resp_h_1,id.resp_h_2,id.resp_h_3,id.resp_h_4
0,dyn,dyn,tcp,S0,S,1,192,168,1,198,185,130,215,13
1,reg,reg,tcp,S0,S,1,192,168,1,198,105,49,30,168
2,reg,reg,tcp,S0,S,1,192,168,1,198,18,206,164,168
3,reg,reg,tcp,S0,S,1,192,168,1,198,181,237,153,227
4,reg,reg,tcp,S0,S,1,192,168,1,198,182,150,31,187


Memory close to full, going to export files first, next script is encoding.ipynb

In [39]:
# horizontal concat df_cat and df_num
df_all = pd.concat([df_cat, df_num], axis=1)

df_all.head()

,id.orig_p,id.resp_p,proto,conn_state,history,target,id.orig_h_1,id.orig_h_2,id.orig_h_3,id.orig_h_4,id.resp_h_1,id.resp_h_2,id.resp_h_3,id.resp_h_4,duration,orig_bytes,resp_bytes,missed_bytes,orig_pkts,orig_ip_bytes,resp_pkts,resp_ip_bytes
0,dyn,dyn,tcp,S0,S,1,192,168,1,198,185,130,215,13,3.141696,0,0,0.0,3.0,180.0,0.0,0.0
1,reg,reg,tcp,S0,S,1,192,168,1,198,105,49,30,168,-,-,-,0.0,1.0,40.0,0.0,0.0
2,reg,reg,tcp,S0,S,1,192,168,1,198,18,206,164,168,-,-,-,0.0,1.0,40.0,0.0,0.0
3,reg,reg,tcp,S0,S,1,192,168,1,198,181,237,153,227,-,-,-,0.0,1.0,40.0,0.0,0.0
4,reg,reg,tcp,S0,S,1,192,168,1,198,182,150,31,187,-,-,-,0.0,1.0,40.0,0.0,0.0


In [40]:
# save the dataframe to csv file
df_all.to_csv('pro/processed.csv', index=False, header=True)